In [19]:
import sys
import mne
import pandas as pd
import os

In [20]:
sys.path.append("..")
from grape.preprocessing_module import Participant

In [21]:
pid = '002'
data_dir = '../data'

Create some empty csv files to save processed data into.

In [22]:
rejection_df = pd.DataFrame(columns=['pid', 'total_epochs', 'dropped_epochs', 'perc_dropped'])
amplitude_latency_df = pd.DataFrame(columns=['pid', 'component', 'amplitude_peak', 'latency_peak'])

cleaned_data_dir = 'cleaned_data'
cleaned_data = 'amplitude_latency_metrics.csv'
rejection_data = 'rejection_metrics.csv'

# Create the full path to the csv file
cleaned_file_path = os.path.join(cleaned_data_dir, cleaned_data)
rejection_file_path = os.path.join(cleaned_data_dir, rejection_data)

# Check if csv files exist or create them
if not os.path.exists(cleaned_file_path):
    amplitude_latency_df.to_csv(cleaned_file_path, index=False)

if not os.path.exists(rejection_file_path):
    rejection_df.to_csv(rejection_file_path, index=False)

Define channels for each region, as well time windows for each region.

In [23]:
N1_channels = ['E22', 'E18', 'E16', 'E10', 'E3', 'E19', 'E11', 'E4', 'E20', 'E12','E5',
               'E26', 'E15', 'E9', 'E2', 'E23', 'E124', 'E118']

N2_channels = ['E26', 'E22', 'E15', 'E9', 'E2', 'E23', 'E18', 'E16', 'E10', 'E3',
               'E24', 'E19', 'E11', 'E4', 'E124', 'E20', 'E12', 'E5', 'E118', 'E13', 
               'E6', 'E112', 'E7', 'E106']


P3_channels = ['E58', 'E65', 'E70', 'E75', 'E83', 'E90', 'E96', 'E51', 'E59', 'E66', 
               'E71', 'E76', 'E84', 'E91', 'E97', 'E47', 'E52', 'E60', 'E67', 'E72', 
               'E77', 'E85', 'E92', 'E98', 'E42', 'E53', 'E61', 'E62', 'E78', 'E86', 
               'E93', 'E37', 'E54', 'COM', 'E79', 'E87', 'E31', 'E55', 'E80', 'VREF']

LPP_channels = P3_channels  # LPP uses the same channels as P3

time_window_N1 = [0.07, 0.15]
time_window_N2 = [0.2, 0.3]
time_window_P3 = [0.35, 0.45]
time_window_LPP = [0.5, 0.8]

Load in data for this participant. 

In [24]:
def load_data(pid, data_dir,
              montage = 'GSN-HydroCel-128', 
              channel_types = {'E127': 'eog', 'E126': 'eog', 'E17': 'eog', 'E15': 'eog', 'E21': 'eog', 'E14': 'eog', 'VREF':'misc'}):
    '''
    Load raw EEG data from the filepath
    '''
    mff_file = os.path.join(data_dir, f'{pid}.mff')
    raw_data = mne.io.read_raw_egi(mff_file, preload=True, verbose=0)
    # Set channel types
    raw_data.set_channel_types(channel_types)    
    # Set montage
    raw_data.set_montage(montage)
    return raw_data

In [25]:
my_data = load_data(pid,data_dir)

/tmp/ipykernel_30180/1216271020.py:10: RuntimeWarning: The unit for channel(s) VREF has changed from V to NA.
  raw_data.set_channel_types(channel_types)
/tmp/ipykernel_30180/1216271020.py:12: RuntimeWarning: Not setting positions of 6 eog channels found in montage:
['E14', 'E15', 'E17', 'E21', 'E126', 'E127']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw_data.set_montage(montage)


In [26]:
# Set verbose="WARNING" to see less output from each method. 
p = Participant(pid, data_dir, verbose="INFO")

out = p.add_raw_data(my_data)
print("RAW DATA SUMMARY:")
out

RAW DATA SUMMARY:


Measurement date,"February 28, 2023 13:44:56 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,125 points
Good channels,"122 EEG, 6 EOG, 1 misc, 2 Stimulus"
Bad channels,None
EOG channels,"E14, E15, E17, E21, E126, E127"
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


In [27]:
out = p.filter_data()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 16501 samples (33.002 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 122 out of 122 | elapsed:    0.6s finished


In [28]:
print("FILTERED DATA SUMMARY:")
out

FILTERED DATA SUMMARY:


Measurement date,"February 28, 2023 13:44:56 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,125 points
Good channels,"122 EEG, 6 EOG, 1 misc, 2 Stimulus"
Bad channels,None
EOG channels,"E14, E15, E17, E21, E126, E127"
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.10 Hz
Lowpass,40.00 Hz


In [29]:
p.find_bad_channels()
p.remove_bad_channels()

There are 27 bad channels found using automated bad channel detection for subject 002
27 bad channels were removed for subject 002


In [30]:
_ = p.fit_ica(method = 'infomax')

64 events found on stim channel STI 014
Event IDs: [1]


64 events found on stim channel STI 014
Event IDs: [1]
Fitting ICA to data using 97 channels (please be patient, this may take a while)
Selecting by number: 15 components
 
Fitting ICA took 7.4s.


In [31]:
p.find_bad_ica()
p.remove_bad_ica()

Effective window size : 0.512 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


There are 4 bad ICA components for subject 002
4 bad ICA components removed for subject 002


In [32]:
out = p.apply_ica()

Applying ICA to Raw instance


    Transforming to ICA space (15 components)
    Zeroing out 4 ICA components
    Projecting back using 97 PCA components


In [33]:
print("POST-ICA SUMMARY:")
out

POST-ICA SUMMARY:


Measurement date,"February 28, 2023 13:44:56 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,125 points
Good channels,"97 EEG, 4 EOG, 1 misc, 2 Stimulus"
Bad channels,None
EOG channels,"E14, E15, E17, E21"
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.10 Hz
Lowpass,40.00 Hz


In [34]:
_ = p.create_epochs()

64 events found on stim channel stim
Event IDs: [1]
Not setting metadata
64 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 64 events and 701 original time points ...
0 bad epochs dropped
EEG channel type selected for re-referencing
Applying average reference.


Applying a custom ('EEG',) reference.


In [35]:
# Create evoked objects for each component
evoked_n1 = p.create_evoked(comp_channels = N1_channels)
evoked_n2 = p.create_evoked(comp_channels = N2_channels)
evoked_p3 = p.create_evoked(comp_channels = P3_channels)
evoked_lpp = p.create_evoked(comp_channels = LPP_channels)

In [36]:
# Find peak amplitude and latency for each component
amp_n1, lat_n1 = p.find_peaks(evoked_n1, time_window_N1)
amp_n2, lat_n2 = p.find_peaks(evoked_n2, time_window_N2)
amp_p3, lat_p3 = p.find_peaks(evoked_p3, time_window_P3)
amp_lpp, lat_lpp = p.find_peaks(evoked_lpp, time_window_LPP)

In [37]:
# Append rejection metrics to DataFrame
new_metrics = pd.DataFrame(
    {'pid': pid,
     'total_epochs': p.total_epochs,
     'dropped_epochs': p.dropped_epochs,
     'perc_dropped': p.perc_dropped},
    index=[0])

rejection_df = pd.concat([rejection_df.astype(new_metrics.dtypes),
                          new_metrics], ignore_index=True)

In [38]:
# Append amplitude and latency metrics to DataFrame for all components
new_metrics = pd.DataFrame(
    {'pid': [pid,pid,pid,pid],
     'component': ['N1','N2','P3','LPP'],
     'amplitude_peak': [amp_n1,amp_n2,amp_p3,amp_lpp],
     'latency_peak': [lat_n1,lat_n2,lat_p3,lat_lpp]},)

amplitude_latency_df = pd.concat([amplitude_latency_df.astype(new_metrics.dtypes),
                                  new_metrics],ignore_index=True)

In [39]:
# Export DataFrames to csv
rejection_df.to_csv(rejection_file_path, mode = 'a', header = True, index=False)
amplitude_latency_df.to_csv(cleaned_file_path, mode = 'a', header = False, index=False)

print('Participant ' + pid + ' processed.')

Participant 002 processed.
